# ตรวจสอบผลการโมเสคด้วยฝนสถานีรายชั่วโมง ตามค่า threshold ทั้งสีวิธีคือ mean,max แบบ apply mfb และไม่ apply mfb  แล้ว aggregrate เป็นค่า matric evalution รายเหตุการณ์ 

In [3]:
'''
2024.09.22
โค้ดนี้พัฒนาโดย รองศาสตราจารย์ ดร. นัฐพล มหาวิค ภาควิชาทรัพยากรธรรมชาติและสิ่งแวดล้อม คณะเกษตรศาสตร์ฯ มหาวิทยาลัยนเรศวร 
ในงานวิจัย เรื่อง "การวิจัยและพัฒนาผลิตภัณฑ์โมเสคฝนประมาณค่าจากเรดาร์ตรวจอากาศในพื้นที่ระดับลุ่มน้ำของประเทศไทยด้วยเทคโนโลยีภูมิสารสนเทศรหัสเปิด"
สนับสนุนทุนวิจัยโดยสํานักงานการวิจัยแห่งชาติ (วช.)  แผนงานการวิจัยและนวัตกรรมแผนงานด้านการบริหารจัดการภัยพิบัติทางธรรมชาติ 
ประจำปีงบประมาณ 2566  ตามสัญญา เลขที่ N25A660467 ผู้นำโค้ดนี้ไปใช้หรือดัดแปลงควรอ้างอิงงานวิจัยชิ้นนี้ตามหลักเกณฑ์การอ้างอิงสากล
เรียนหลักการเรดาร์และภูมิสารสนเทศ ที่ https://www.youtube.com/@Nattapon_Mahavik/playlists
หนังสือเรดาร์ตรวจอากาศทางอุตุนิยมวิทยา สำนักพิมพ์จุฬาฯ : https://www.chulabook.com/education/144567
หนังสือออนไลน์เรดาร์ตรวจอากาศทางอุตุนิยมวิทยา สำนักพิมพ์จุฬาฯ : https://www.chulabook.com/education/205129
ติดต่อ nattaponm@nu.ac.th
'''
#ตรวจสอบผลการโมเสคด้วยฝนสถานีรายชั่วโมง ตามค่า threshold ทั้งสีวิธีคือ mean,max แบบ apply mfb และไม่ apply mfb  แล้ว aggregrate เป็นค่า matric evalution รายเหตุการณ์ 
import os
import numpy as np
import pandas as pd
import rasterio
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from collections import defaultdict
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def read_mosaic_data(file_path):
    with rasterio.open(file_path) as src:
        return src.read(1), src.transform

def extract_mosaic_rainfall(lon, lat, mosaic_data, transform):
    row, col = rasterio.transform.rowcol(transform, lon, lat)
    return mosaic_data[row, col]

def calculate_metrics(observed, predicted):
    if len(observed) < 2:
        return {metric: np.nan for metric in ['MSE', 'RMSE', 'R-squared', 'MAE', 'NSE', 'PBIAS', 'Correlation', 'MFB', 'Data_Count']}
    
    mse = mean_squared_error(observed, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(observed, predicted)
    mae = mean_absolute_error(observed, predicted)
    
    try:
        nse = 1 - (np.sum((observed - predicted)**2) / np.sum((observed - np.mean(observed))**2))
    except ZeroDivisionError:
        nse = np.nan
    
    try:
        pbias = np.sum(predicted - observed) / np.sum(observed) * 100
    except ZeroDivisionError:
        pbias = np.nan
    
    correlation = np.corrcoef(observed, predicted)[0, 1]
    
    try:
        mfb = np.mean(observed) / np.mean(predicted) # ถ้าค่า mfb > 1 ฝนเรดาร์ประมาณค่าได้น้อยกว่าฝนภาคพื้นดิน
    except ZeroDivisionError:
        mfb = np.nan
    
    return {
        'MSE': mse,
        'RMSE': rmse,
        'R-squared': r2,
        'MAE': mae,
        'NSE': nse,
        'PBIAS': pbias,
        'Correlation': correlation,
        'MFB': mfb,
        'Data_Count': len(observed)
    }

def process_data(mosaic_base_dir, gauge_dir, match_files_dir, thresholds):
    all_results = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))
    
    match_file_path = os.path.join(match_files_dir, 'matched_rainfall_files.csv')
    
    if not os.path.exists(mosaic_base_dir) or not os.path.exists(gauge_dir) or not os.path.exists(match_file_path):
        print("Warning: Required directories or match file not found")
        return all_results
    
    match_data = pd.read_csv(match_file_path)
    
    mosaic_methods = ['1mean_mos', '2max_mos']
    mfb_methods = ['0mfb', '0no_mfb']
    
    for mosaic_method in mosaic_methods:
        for mfb_method in mfb_methods:
            mosaic_dir = os.path.join(mosaic_base_dir, mosaic_method, '0mosaics', mfb_method)
            method_name = f"{mosaic_method}_{mfb_method}"
            
            filtered_data_by_threshold = {threshold: [] for threshold in thresholds}
            
            for _, row in tqdm(match_data.iterrows(), desc=f"Processing {method_name}", total=len(match_data)):
                radar_file_path = os.path.join(mosaic_dir, row['Radar_File'])
                gauge_file_path = os.path.join(gauge_dir, row['Gauge_File'])
                
                if not os.path.exists(radar_file_path) or not os.path.exists(gauge_file_path):
                    print(f"Warning: File not found for {method_name}: {row['Radar_File']} or {row['Gauge_File']}")
                    continue
                
                try:
                    mosaic_data, transform = read_mosaic_data(radar_file_path)
                    gauge_data = pd.read_csv(gauge_file_path)
                    
                    gauge_data['mosaic_rainfall'] = gauge_data.apply(
                        lambda x: extract_mosaic_rainfall(x['longitude'], x['latitude'], mosaic_data, transform),
                        axis=1
                    )
                    
                    for threshold in thresholds:
                        hourly_filtered = gauge_data[
                            (gauge_data['rain'] > threshold) & 
                            (gauge_data['mosaic_rainfall'] > threshold)
                        ]
                        if not hourly_filtered.empty:
                            filtered_data_by_threshold[threshold].append(hourly_filtered)
                
                except Exception as e:
                    print(f"Error processing data for {method_name} - {row['Radar_File']}: {str(e)}")
                    continue
            
            for threshold in thresholds:
                if filtered_data_by_threshold[threshold]:
                    aggregated_data = pd.concat(filtered_data_by_threshold[threshold])
                    
                    event_data = aggregated_data.groupby(['station_code', 'longitude', 'latitude'])[['rain', 'mosaic_rainfall']].sum().reset_index()
                    
                    if len(event_data) > 0:
                        metrics = calculate_metrics(event_data['rain'], event_data['mosaic_rainfall'])
                    else:
                        metrics = {'Data_Count': 0}
                    
                    all_results[method_name][threshold]['full_event'] = metrics
                else:
                    all_results[method_name][threshold]['full_event'] = {'Data_Count': 0}

    return all_results

def save_detailed_results(results, output_dir):
    ensure_dir(output_dir)
    
    all_data = []
    for method in results:
        for threshold in results[method]:
            metrics = results[method][threshold]['full_event']
            row = {
                'Method': method,
                'Threshold': threshold,
                'Event': 'full_event'
            }
            row.update(metrics)
            all_data.append(row)
    
    df = pd.DataFrame(all_data)
    df.to_csv(os.path.join(output_dir, 'detailed_results_mosaic.csv'), index=False)

def create_summary(results):
    summary = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))
    
    for method in results:
        for threshold in results[method]:
            summary[method][threshold] = results[method][threshold]['full_event']
    
    return summary

def save_summary(summary, output_dir):
    ensure_dir(output_dir)
    
    all_data = []
    for method in summary:
        for threshold in summary[method]:
            row = {
                'Method': method,
                'Threshold': threshold
            }
            row.update(summary[method][threshold])
            all_data.append(row)
    
    df = pd.DataFrame(all_data)
    df.to_csv(os.path.join(output_dir, 'summary_results_mosaic.csv'), index=False)
    
    print("Summary of Results:")
    print(df.to_string())

def main():
    mosaic_base_dir = '../00run_batch_acchr_codes/2output/0Hourly/0Sontihn_dbz_single/0mosaic'
    gauge_dir = './0Zprocessing_data/0hourly_rain_thai_mos/0Sontihn/0Hourly/'
    match_files_dir = './0Zprocessing_data/0matched_files_mos_thai/'
    output_dir = './0Zprocessing_data/0analysis_validate_mos_results/'
    
    ensure_dir(output_dir)
    
    thresholds = [0, 0.1, 0.5, 1, 2, 5, 10]

    results = process_data(mosaic_base_dir, gauge_dir, match_files_dir, thresholds)
    save_detailed_results(results, output_dir)

    summary = create_summary(results)
    save_summary(summary, output_dir)

    print(f"\nDetailed results and summary saved in {output_dir}")

if __name__ == "__main__":
    main()

Processing 2max_mos_0no_mfb: 100%|███████████████████████████████████████████████████| 240/240 [01:01<00:00,  3.91it/s]

Summary of Results:
               Method  Threshold          MSE       RMSE  R-squared        MAE       NSE      PBIAS  Correlation       MFB  Data_Count
0      1mean_mos_0mfb        0.0  1502.114370  38.757120  -0.350612  26.106083 -0.350612 -79.702649     0.715349  4.926751       386.0
1      1mean_mos_0mfb        0.1  1290.690136  35.926176  -0.276553  23.723523 -0.276553 -78.301780     0.715933  4.608673       375.0
2      1mean_mos_0mfb        0.5   472.347267  21.733552  -0.281596  14.384447 -0.281596 -75.731017     0.655300  4.120486       244.0
3      1mean_mos_0mfb        1.0   203.888657  14.278959  -0.306860   9.501089 -0.306860 -71.653938     0.571110  3.527827       138.0
4      1mean_mos_0mfb        2.0    91.197622   9.549745  -0.728465   6.943965 -0.728465 -64.672550     0.212884  2.830660        44.0
5      1mean_mos_0mfb        5.0          NaN        NaN        NaN        NaN       NaN        NaN          NaN       NaN         NaN
6      1mean_mos_0mfb       10.0   